In [16]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import matplotlib.image as mpimg
from mayavi import mlab
mlab.init_notebook('x3d')
from matplotlib import rc
import pandas as pd
rc('font',**{'family':'serif','serif':['Times New Roman']})
rc('text', usetex=True)
rc('font', size=14)

Notebook initialized with x3d backend.


## Objective

(1) Gauss-Newton V.S. BFGS
5 methods are used to compare with each other:


| method       | Gradient   | Flood    | Iteration      |
|--------------|------------|----------|----------------|
| ZY           | numerical  | fast     |  Gauss-Newton  |
| LemoineGN    | analytic   | none     |  Gauss-Newton  |
| LemoineBFGS  | analytic   | none     |  BFGS          |
| LemoineGeo   | numerical  | geometry |  BFGS          |
| Sussman      | numerical  | geometry |  Gauss-Newton  |

Lemoine's analytic gradient is fast, can be 200 faster than the numerical gradient from geometry algorithm.
We propose a fast flood algorithm, even though uses a numerical gradient, the computational time is compatable with lemoine's analytic gradient while keep rubustness.

(2) Initial guess

Focus on:

- effeciency
- accuracy
- robustness

In [85]:
case_path = ['extreme',
        'linear',
        'exponential',
       ]
mathod_list = [
    'Lemoine, Gauss-Newton_info',
    'Lemoine, BFGS, numerical_info',
    'Lemoine, BFGS, analytic_info',
    'Lemoine, Gauss-Newton_old_info',
    'Lemoine, BFGS, numerical_old_info',
    'Lemoine, BFGS, analytic_old_info'
]

In [102]:
dics = {}
for p in case_path:
    dics.update({p:{}})
    for method in mathod_list:
        casename = method
        if 'Gauss' in method:
            casename = 'GN'
        if 'analytic' in method:
            casename = 'BFGS_ana'
        if 'numerical' in method:
            casename = 'BFGS_num'
        if 'old' in method:
            casename = casename + '_old'
        dics[p].update({casename:{}})
        if 'old' in method:
            dics[p][casename].update({'initial_guess':'old'}) 
        else:
            dics[p][casename].update({'initial_guess':' '}) 
        file = open(p+'/'+method+'.dat')
        for line in file:
            if 'time' in line:
                cpu_time = float(line.split('=')[-1])
                dics[p][casename].update({'cpu_time':cpu_time}) 
            if 'num_iter1' in line:
                num_iter1 = float(line.split('=')[-1])
                dics[p][casename].update({'num_iter1':num_iter1}) 
            if 'num_iter2' in line:
                num_iter2 = float(line.split('=')[-1])
                dics[p][casename].update({'num_iter2':num_iter2}) 
            if 'angle_error' in line:
                angle_error = float(line.split('=')[-1])
                dics[p][casename].update({'angle_error':angle_error}) 
            if 'centroid_error' in line:
                centroid_error = float(line.split('=')[-1])
                dics[p][casename].update({'centroid_error':centroid_error}) 
        file.close()

In [107]:
cases = []
frames = []

for case in dics:
    cases.append(case)
    frames.append(pd.DataFrame.from_dict(dics[case], orient='index'))
    
linear = pd.DataFrame.from_dict(dics['linear'], orient='index')
extreme = pd.DataFrame.from_dict(dics['extreme'], orient='index')
exponential = pd.DataFrame.from_dict(dics['exponential'], orient='index')

dat = pd.concat(frames, keys=cases)
pd.DataFrame.to_csv(dat,'table.csv')

In [108]:
linear

,initial_guess,cpu_time,num_iter1,num_iter2,angle_error,centroid_error
GN,,0.109201,3.10372,0.00000,6.692240e-09,2.036865e-09
BFGS_num,,96.069684,6.32153,12.49048,1.115103e-08,2.850355e-09
BFGS_ana,,0.431708,6.32156,12.49076,1.050430e-08,2.809178e-09
GN_old,old,0.114424,3.26855,0.00000,2.448042e-13,7.000087e-06
BFGS_num_old,old,100.267147,6.43482,12.97012,1.505826e-08,2.890787e-09
BFGS_ana_old,old,0.427662,6.29313,12.16986,1.720482e-09,2.284442e-09


In [109]:
extreme

,initial_guess,cpu_time,num_iter1,num_iter2,angle_error,centroid_error
GN,,0.098996,3.18405,0.00000,4.286675e-15,2.552598e-09
BFGS_num,,95.652586,6.71862,14.51969,1.257405e-13,3.498057e-09
BFGS_ana,,0.416248,6.69491,13.28435,2.019057e-13,2.551983e-09
GN_old,old,0.115287,4.34207,0.00000,1.769471e-08,7.175986e-06
BFGS_num_old,old,103.260843,7.14457,16.32459,1.257405e-13,3.680531e-09
BFGS_ana_old,old,0.445468,7.12058,15.08992,2.019057e-08,2.736136e-09


In [110]:
exponential

,initial_guess,cpu_time,num_iter1,num_iter2,angle_error,centroid_error
GN,,0.103028,2.40359,0.00000,1.071365e-14,1.982331e-09
BFGS_num,,84.585275,5.69971,10.85868,2.132465e-10,2.416453e-09
BFGS_ana,,0.422287,5.69968,10.85865,2.128264e-10,2.407579e-09
GN_old,old,0.102545,2.40360,0.00000,1.071365e-14,1.982330e-09
BFGS_num_old,old,81.224210,5.69975,10.85866,2.132465e-10,2.416462e-09
BFGS_ana_old,old,0.421157,5.38376,10.43183,1.577744e-08,2.078119e-09
